In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from project_config import ProjectConfig
pcfg = ProjectConfig()

from tqdm import tqdm
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
NPARAMS_THRESHOLD = 2276356

root_dir = Path(pcfg.MMAGIC_MODEL_CONFIGS_DIR_PATH).expanduser()
subdirs = [
    "srcnn",
    "srgan_resnet",
    "edsr",
    "esrgan",
    "rdn",
    "dic",
    "ttsr",
    "glean",
    "liif",
    "real_esrgan",
    "swinir",
]
oom_config_names = [
    "dic_gan-x8c48b6_4xb2-500k_celeba-hq.py", #oom
    "dic_x8c48b6_4xb2-150k_celeba-hq.py", #oom
]
unsupported_config_methodsss =[
    "ttsr", #TypeError: forward() missing 2 required positional arguments: 'soft_attention' and 'textures'
    "glean", #input size assertion 32
    "liif", #TypeError: forward() missing 2 required positional arguments: 'coord' and 'cell'
]

from collections import defaultdict
model_methods_analyses = defaultdict(
    lambda: defaultdict(
        lambda: defaultdict(
            dict
        )
    )
)

In [3]:
from utils.analyze_models import inference_nflops_nparams, to_dict,\
    filter_dict_by_flag, convert_size_to_int, save_to_py, run_model_in_subprocess


def inference_sr_nflops_nparams_wrapper(model_cfg_path: str):
    inputs = [
        model_cfg_path,
        '--shape',
        '3',
        '128',
        '128',
    ]
    return run_model_in_subprocess(inputs)
    

In [4]:
all_model_cfg_paths = list()
for subdir in subdirs:
    subdir_path = root_dir / subdir

    if subdir_path.exists() and subdir_path.is_dir():
        model_cfg_paths = list(subdir_path.glob("*.py"))
        all_model_cfg_paths.extend(model_cfg_paths)
    else:
        logger.warning(f"Subdirectory {subdir} does not exist or is not a directory.")

In [5]:
grayscale_cfg_names = [str(model_cfg_path.name) for model_cfg_path in all_model_cfg_paths if 'gray' in str(model_cfg_path.name)]
grayscale_cfg_names

['swinir_s126w7d6e180_8xb1-lr2e-4-1600k_dfwb-grayCAR40.py',
 'swinir_s126w7d6e180_8xb1-lr2e-4-1600k_dfwb-grayCAR30.py',
 'swinir_s126w7d6e180_8xb1-lr2e-4-1600k_dfwb-grayCAR20.py',
 'swinir_s128w8d6e180_8xb1-lr2e-4-1600k_dfwb-grayDN15.py',
 'swinir_s126w7d6e180_8xb1-lr2e-4-1600k_dfwb-grayCAR10.py',
 'swinir_s128w8d6e180_8xb1-lr2e-4-1600k_dfwb-grayDN25.py',
 'swinir_s128w8d6e180_8xb1-lr2e-4-1600k_dfwb-grayDN50.py']

In [6]:
for model_cfg_path in tqdm(all_model_cfg_paths, desc="Analyzing all model config", unit="file"):
    model_method = model_cfg_path.parent.name
    model_cfg_name = model_cfg_path.name
    
    if model_method not in unsupported_config_methodsss:
        if model_cfg_name not in oom_config_names:
            if model_cfg_name not in grayscale_cfg_names:
                # logger.info(f"Analyzing {model_method}/{model_cfg_name}...")
                print(f"\nAnalyzing {model_method}/{model_cfg_name}...")
                flops, params = inference_sr_nflops_nparams_wrapper(str(model_cfg_path))
                nparams = convert_size_to_int(params)

                model_methods_analyses[model_method][model_cfg_name]["flops"] = flops
                model_methods_analyses[model_method][model_cfg_name]["params"] = params
                model_methods_analyses[model_method][model_cfg_name]["flag"] = True if nparams <= NPARAMS_THRESHOLD else False
            else:
                print(f"\nOnly for Grayscale, Skip {model_method}/{model_cfg_name}...")
        else:
            print(f"\nOOM, Skip {model_method}/{model_cfg_name}...")
    else:
        print(f"\nUnsupported Method, Skip {model_method}/{model_cfg_name}...")

Analyzing all model config:   0%|          | 0/55 [00:00<?, ?file/s]


Analyzing srcnn/srcnn_x4k915_1xb16-1000k_div2k.py...
Input shape: (3, 128, 128)
Flops: 5.243G
Params: 20.099K



Analyzing all model config:   2%|▏         | 1/55 [00:12<11:12, 12.45s/file]


Analyzing srgan_resnet/msrresnet_x4c64b16_1xb16-1000k_div2k.py...
Input shape: (3, 128, 128)
Flops: 41.555G
Params: 1.518M



Analyzing all model config:   4%|▎         | 2/55 [00:23<10:19, 11.69s/file]

...

Analyzing all model config:  95%|█████████▍| 52/55 [08:01<00:40, 13.50s/file]


Analyzing swinir/swinir_gan-x2s64w8d6e180_8xb4-lr1e-4-600k_df2k-ost.py...
Input shape: (3, 128, 128)
Flops: 0.208T
Params: 11.679M



Analyzing all model config:  96%|█████████▋| 53/55 [08:15<00:27, 13.85s/file]


Only for Grayscale, Skip swinir/swinir_s128w8d6e180_8xb1-lr2e-4-1600k_dfwb-grayDN50.py...

Analyzing swinir/swinir_psnr-x4s64w8d9e240_8xb4-lr1e-4-600k_df2k-ost.py...
Input shape: (3, 128, 128)
Flops: 0.505T
Params: 28.013M



Analyzing all model config: 100%|██████████| 55/55 [08:34<00:00,  9.35s/file]


In [7]:
model_methods_analyses_dict = to_dict(model_methods_analyses)
filtered_model_methods_analyses_dict = filter_dict_by_flag(model_methods_analyses_dict, key="flag", target_value=True)

save_to_py(
    pcfg.SRC_MODELS_ANALYSES_FILE_PATH,
    "model_methods_analyses",
    model_methods_analyses_dict
)

save_to_py(
    pcfg.SRC_FILTERED_MODELS_ANALYSES_FILE_PATH,
    "filtered_model_methods_analyses",
    filtered_model_methods_analyses_dict
)